In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [3]:
options = {
 "WLSACCESSID":"a4353fb7-f95b-4075-b288-ca3f60983b36",
"WLSSECRET":"d894d460-2dac-4210-8c40-c91c68ecfb13",
"LICENSEID":2562382
}

In [72]:
# Number of tasks and workers
num_tasks = 200
num_workers = 200

# Random cost matrix
np.random.seed(1)
costs = - np.random.randint(1, 10, (num_tasks, num_workers))

# Create the environment with license parameters
with gp.Env(params=options) as env:
    # Create a Gurobi model within the environment
    with gp.Model(env=env) as model:
        
        # Create variables: match[i, j] = 1 if task i is assigned to worker j, otherwise 0
        match = model.addVars(num_tasks, num_workers, name="x")
        match.start = np.zeros((num_tasks, num_workers), dtype=object)
        # match.start = solution.reshape((num_tasks, num_workers))
        # Set objective: minimize total assignment cost
        model.setObjective(gp.quicksum(costs[i][j] * match[i, j] for i in range(num_tasks) for j in range(num_workers)), GRB.MINIMIZE)
        
        # Constraints: each task is assigned to exactly one worker
        for i in range(num_tasks):
            model.addConstr(gp.quicksum(match[i, j] for j in range(num_workers)) <= 1, f"task_{i}")
        
        # Constraints: each worker is assigned to exactly one task
        for j in range(num_workers):
            model.addConstr(gp.quicksum(match[i, j] for i in range(num_tasks)) <= 1, f"worker_{j}")
        
        # Optimize the model
        model.optimize()
        solution = np.array(model.x)


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562382
Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 24.0.0 24A335)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
Optimize a model with 400 rows, 40000 columns and 80000 nonzeros
Model fingerprint: 0xb7aa134b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 400 rows, 40000 columns, 80000 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 4.000e+04
 Factor NZ  : 8.020e+04 (roughly 17 MB of memory)
 Factor Ops : 2.

In [73]:
np.shape(solution)

(40000,)

In [74]:
solution.sum()

200.0

In [75]:
-(costs.flatten() * solution).sum()

1800.0